<a href="https://colab.research.google.com/github/Tewf/Tewf/blob/main/knapSack_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
import pandas as pd
import gradio as gr
import numpy as np
import math
import joblib
import datasets

In [58]:
#algorithm from google
def knapSack(W, wt, val, n):
  
    # Base Case
    if n == 0 or W == 0 :
        return 0
  
    # If weight of the nth item is more than Knapsack of capacity
    # W, then this item cannot be included in the optimal solution
    if (wt[n-1] > W):
        return knapSack(W, wt, val, n-1)
  
    # return the maximum of two cases:
    # (1) nth item included
    # (2) not included
    else:
        return max(val[n-1] + knapSack(W-wt[n-1], wt, val, n-1),
                   knapSack(W, wt, val, n-1))
  
# end of function knapSack
  
# To test above function


In [59]:
val = [60, 100, 120]
wt = [10, 20, 30]
W = 50
n = len(val)
print(knapSack(W, wt, val,n))

220


In [60]:
#the solution
val = [60, 100, 120]
wt = [10, 20, 30]
kuri= ["Banne", "onion", "pomme"]
W = 50
data={"object name":kuri,"weight":wt,"value":val}
df= pd.DataFrame(data)
df=df.sort_values(by=['value'], ascending=False)
df


,object name,weight,value
2,pomme,30,120
1,onion,20,100
0,Banne,10,60


In [61]:
#algo solution
def Fas(i,a,k,v,y,b):
  x=i-v
  b=x//y
  a=(W//df["weight"].loc[df.index[k]])-b
  while a<0:
    k=k+1
    v=v+W//df["weight"].loc[df.index[k-1]]
    y=y-1
    x=i-v
    b=x//y
    a=(W//df["weight"].loc[df.index[k]])-b
  return a*df["weight"].loc[df.index[k]]+b*df["weight"].loc[df.index[k+1]]+df["weight"].loc[df.index[k+(x%y)]],a,k,v,y,b

In [62]:
#secant 
def secant(i,a,k,v,y,b):
  while Fas(i,a,k,v,y,b)[0]<=Fas(i+1,a,k,v,y,b)[0]:
    i=i+1
    a,k,v,y,b=Fas(i,a,k,v,y,b)[1],Fas(i,a,k,v,y,b)[2],Fas(i,a,k,v,y,b)[3],Fas(i,a,k,v,y,b)[4],Fas(i,a,k,v,y,b)[5]
  i1 = math.floor(i + (W-Fas(i,a,k,v,y,b)[0])/(Fas(i+1,a,k,v,y,b)[0]-Fas(i,a,k,v,y,b)[0]))
  if Fas(i,a,k,v,y,b)[0]<=Fas(i1,a,k,v,y,b)[0] or Fas(i1,a,k,v,y,b)[0]<W :
    i1=i+1
  i=i1
  return i

In [63]:
i,k,v,y,b=0,0,0,len(val)-1,0
a=(W//df["weight"].loc[df.index[k]])-b
while Fas(i,a,k,v,y,b)[0]>W:
  i=i+1
  a,k,v,y,b=Fas(i,a,k,v,y,b)[1],Fas(i,a,k,v,y,b)[2],Fas(i,a,k,v,y,b)[3],Fas(i,a,k,v,y,b)[4],Fas(i,a,k,v,y,b)[5]
  print(Fas(i,a,k,v,y,b,))

(50, 1, 0, 0, 2, 0)


In [64]:
i,k,v,y,b=0,0,0,len(val)-1,0
W=70
a=(W//df["weight"].loc[df.index[k]])-b
while Fas(i,a,k,v,y,b)[0]>W:
  i=secant(i,a,k,v,y,b)
  a,k,v,y,b=Fas(i,a,k,v,y,b)[1],Fas(i,a,k,v,y,b)[2],Fas(i,a,k,v,y,b)[3],Fas(i,a,k,v,y,b)[4],Fas(i,a,k,v,y,b)[5]
  print(Fas(i,a,k,v,y,b,))

(80, 1, 0, 0, 2, 1)
(70, 1, 0, 0, 2, 1)


In [65]:
#final solution
def team(l,W):
  l=l.sort_values(by=['value'], ascending=False)
  i,k,v,y,b=0,0,0,len(l.index)-1,0
  a=(W//l["weight"].loc[l.index[k]])-b
  while Fas(i,a,k,v,y,b)[0]>W:
    i=secant(i,a,k,v,y,b)
    a,k,v,y,b=Fas(i,a,k,v,y,b)[1],Fas(i,a,k,v,y,b)[2],Fas(i,a,k,v,y,b)[3],Fas(i,a,k,v,y,b)[4],Fas(i,a,k,v,y,b)[5]
  h=i-v
  return Fas(i,a,k,v,y,b)[0],a*l["value"].loc[l.index[k]]+b*l["value"].loc[l.index[k+1]]+l["value"].loc[l.index[k+(h%y)]],str(a)+l["object name"].loc[l.index[k]]+" "+str(b)+l["object name"].loc[l.index[k+1]]+" 1"+l["object name"].loc[l.index[k+(h%y)]]

In [75]:
team(df,70)

,object name,weight,value
2,pomme,30,120
1,onion,20,100
0,Banne,10,60


In [76]:
input=gr.Dataframe(
            headers=["object name", "weight", "value"],
            datatype=["str", "number", "number"],
            row_count=3,
            col_count=(3, "fixed"))
#outputs = [gr.Dataframe(row_count = (1, "dynamic"), col_count=(2, "fixed"), label="Predictions", headers=["best combination","best value"])]
demo=gr.Interface(fn=team, inputs=[input,gr.Number(label="bag weight")],outputs=[gr.Number(label="maximum weight"),gr.Number(label="best value"),gr.Textbox(label="Best Combination")],title="KnapSack Solver",examples=[[df]])
demo.launch(debug=True,share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a06b40b86070f9b3ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a06b40b86070f9b3ae.gradio.live
